In [69]:
from bs4 import BeautifulSoup as bs
from requests import get
import requests
import re


Journal of Neurophysiology is structured such that you can find all volume and issue information hidden on any of the volume pages, so we can just start with the first volume and create a list of links for the remaining volumes.

In [ ]:
start_url = 'https://journals.physiology.org/loi/jn/group/d1940.y1940'
url = start_url
year = get(url)
year_page = bs(year.text, 'html.parser')
all_vol = year_page.find_all('a',{'href': re.compile(r'/toc/jn/')})
vol_list = []
for v in all_vol:
    try: 
        vol_str = re.search(r'Volume\s\d{1,3}',str(v)).group()
        vol_int = int(vol_str[7:])
        vol_list.append(vol_int)
    except AttributeError:
        print('Something bad happened.')

vol_list.sort()


Default digital open access starts in Volume 77, issue 1. Perhaps start the loop from this point instead of going through all of the earlier volumes, which are all paywalled and in PDF form.

In [ ]:
open_access = [i for i in vol_list if i>76]
print(open_access)

In [ ]:
last_v = 0
base_url = 'https://journals.physiology.org/toc/jn/'
for v in open_access:
    # count the issues in each volume
    if v!=last_v:
        issue=1
    else:
        issue+=1
    
    # construct the url for the issue and load it
    issue_url = base_url + str(v) +'/' + str(issue)
    print(issue_url)
    get_issue = get(issue_url)
    issue_page = bs(get_issue.text, 'html.parser')
    issue_date = issue_page.find('div',{'class':'col-sm-4 gray-bg toc-right-side'}).\
        find('span',{'class':'coverDate'}).get_text()

    # scrape all of the articles in this issue
    toc = issue_page.find_all('div',{'class':'table-of-content'})[0].find_all('div',{'class':'issue-item'})

    # loop through the articles
    for c in toc:
        
        # check if article is behind a paywall. If so, skip it (for now)
        if not c.find('div',{'class':'badges'}).get_text():
            continue
        
        # get article metadata
        art_url = 'https://journals.physiology.org' + c.find('a').get('href')
        art_title = c.find('h4').text
        auth = c.find('ul',{'class':'rlist--inline loa'}).find_all('li')
        art_authors = [a.get_text().replace(', and ','') for a in auth]
        art_doi = c.select(".epub-section__item")[0].find('a').get('href')[16:]
        
        # load article
        get_art = get(art_url)
        art_page = bs(get_art, 'html.parser')

        # get abstract
        # THIS IS WHERE I STARTED TO WONDER HOW TO STRUCTURE THE DATABASE
            
        
    
    

# Testing stuff out

In [39]:
page = get('https://journals.physiology.org/toc/jn/123/1')
test_page = bs(page.text, 'html.parser')
toc = test_page.find_all('div',{'class':'table-of-content'})[0].find_all('div',{'class':'issue-item'})
toc[0].find('div',{'class':'badges'}).get_text()

''

In [37]:
date = test_page.find('div',{'class':'col-sm-4 gray-bg toc-right-side'}).\
    find('span',{'class':'coverDate'}).get_text()

In [38]:
date

'Dec 2014'

In [43]:
toc[0].select(".epub-section__item")[0].find('a').get('href')[16:]

'10.1152/jn.00537.2019'